In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import os
from os.path import join
import re
import time
import argparse
import pickle
import ldspec
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
PGEN_FILE='/n/scratch/users/j/jz286/imp_geno_chimp/ukb_imp_chr@_v3_chimp'
ANNOT_FILE='/home/jz286/WES_analysis/LDSPEC/experiments/job.analysis_imp_geno_chimp/reg_annot_file/reg_annot_file.prox_gene_fct_all_ld.txt'

PGEN_NEW_FILE='/n/scratch/users/j/jz286/imp_geno_rep287K_chimp/ukb_imp_chr@_v3_chimp'
OUT_FOLDER='/n/groups/price/martin/LDSPEC_data/annot_rep287K'
OUT_ANNOT_FILE='/home/jz286/WES_analysis/LDSPEC/experiments/job.analysis_imp_geno_chimp/reg_annot_file/reg_annot_file.rep287K.prox_gene_fct_all_ld.txt'

# PGEN_FILE='/n/scratch/users/j/jz286/imp_geno_chimp/ukb_imp_chr22_v3_chimp'
# PGEN_NEW_FILE='/n/scratch/users/j/jz286/imp_geno_rep287K_chimp/ukb_imp_chr22_v3_chimp'

In [3]:
# PGEN_FILE
print("# Loading PGEN_FILE")
dic_data = {}
if "@" not in PGEN_FILE:  # Load single CHR
    temp_dic = ldspec.util.read_pgen(PGEN_FILE)
    dic_data[temp_dic["pvar"]["CHR"][0]] = temp_dic.copy()
else:
    for CHR in range(1, 23):  # Check all 23 CHRs
        if os.path.exists(PGEN_FILE.replace("@", "%s" % CHR) + ".pgen"):
            dic_data[CHR] = ldspec.util.read_pgen(
                PGEN_FILE.replace("@", "%s" % CHR)
            )
for CHR in dic_data:
    n_sample = dic_data[CHR]["psam"].shape[0]
    n_snp = dic_data[CHR]["pvar"].shape[0]
    print("    CHR%2d: %d samples, %d SNPs" % (CHR, n_sample, n_snp))
    
# PGEN_NEW_FILE            
print("# Loading PGEN_NEW_FILE")
dic_data_new = {}
if "@" not in PGEN_NEW_FILE:  # Load single CHR
    temp_dic = ldspec.util.read_pgen(PGEN_NEW_FILE)
    dic_data_new[temp_dic["pvar"]["CHR"][0]] = temp_dic.copy()
else:
    for CHR in range(1, 23):  # Check all 23 CHRs
        if os.path.exists(PGEN_NEW_FILE.replace("@", "%s" % CHR) + ".pgen"):
            dic_data_new[CHR] = ldspec.util.read_pgen(
                PGEN_NEW_FILE.replace("@", "%s" % CHR)
            )
for CHR in dic_data_new:
    n_sample = dic_data_new[CHR]["psam"].shape[0]
    n_snp = dic_data_new[CHR]["pvar"].shape[0]
    print("    CHR%2d: %d samples, %d SNPs" % (CHR, n_sample, n_snp))

# Loading PGEN_FILE
    CHR 1: 337426 samples, 1161341 SNPs
    CHR 2: 337426 samples, 1259312 SNPs
    CHR 3: 337426 samples, 1059605 SNPs
    CHR 4: 337426 samples, 1074447 SNPs
    CHR 5: 337426 samples, 964806 SNPs
    CHR 6: 337426 samples, 976090 SNPs
    CHR 7: 337426 samples, 868301 SNPs
    CHR 8: 337426 samples, 826076 SNPs
    CHR 9: 337426 samples, 641608 SNPs
    CHR10: 337426 samples, 747773 SNPs
    CHR11: 337426 samples, 730385 SNPs
    CHR12: 337426 samples, 705493 SNPs
    CHR13: 337426 samples, 537961 SNPs
    CHR14: 337426 samples, 482519 SNPs
    CHR15: 337426 samples, 423991 SNPs
    CHR16: 337426 samples, 465661 SNPs
    CHR17: 337426 samples, 406065 SNPs
    CHR18: 337426 samples, 422346 SNPs
    CHR19: 337426 samples, 336680 SNPs
    CHR20: 337426 samples, 329864 SNPs
    CHR21: 337426 samples, 200644 SNPs
    CHR22: 337426 samples, 199680 SNPs
# Loading PGEN_NEW_FILE
    CHR 1: 287048 samples, 1161345 SNPs
    CHR 2: 287048 samples, 1259298 SNPs
    CHR 3: 287

In [4]:
print("# Loading --annot_file")
dic_annot_path = {}
dic_pannot_path = {}

annot_file_list = []
CHR0 = list(dic_data)[0]
if ANNOT_FILE.endswith(".txt"):
    with open(ANNOT_FILE, "r") as f:
        for line in f:
            line = line.strip()
            if os.path.exists(line.replace("@", "%d" % CHR0)):
                annot_file_list.append(line)
            else:
                print("    Skip: %s" % line)
else:
    for line in ANNOT_FILE.split(","):
        line = line.strip()
        if os.path.exists(line.replace("@", "%d" % CHR0)):
            annot_file_list.append(line)
        else:
            print("    Skip: %s" % line)

for annot_file in annot_file_list:
    annot_file = annot_file.strip()
    if annot_file.endswith((".annot.gz", ".pannot_mat.npz")) is False:
        print("    Skip: %s" % annot_file)
        continue
    annot_name = ldspec.util.get_annot_name_from_file(annot_file)
    if annot_file.endswith(".annot.gz"):
        dic_annot_path[annot_name] = {}
        for CHR in dic_data:
            fpath = annot_file.replace("@", "%d" % CHR)
            if os.path.exists(fpath):
                dic_annot_path[annot_name][CHR] = fpath
        CHR_set_annot = set(dic_annot_path[annot_name])
    if annot_file.endswith(".pannot_mat.npz"):
        dic_pannot_path[annot_name] = {}
        for CHR in dic_data:
            fpath = annot_file.replace("@", "%d" % CHR)
            if os.path.exists(fpath):
                dic_pannot_path[annot_name][CHR] = fpath
        CHR_set_annot = set(dic_pannot_path[annot_name])

# Check: all annots and pannots have the same set of CHRs
for annot_name in dic_annot_path:
    err_msg = "Set of CHRs does not match for %s" % annot_name
    assert set(dic_annot_path[annot_name]) == CHR_set_annot, err_msg
for annot_name in dic_pannot_path:
    err_msg = "Set of CHRs does not match for %s" % annot_name
    assert set(dic_pannot_path[annot_name]) == CHR_set_annot, err_msg
print(
    "    Detected %d CHRs for all files: %s"
    % (len(CHR_set_annot), ",".join(["%d" % x for x in CHR_set_annot]))
)

# Check: annots have the same col_list across CHRs
for annot_name in dic_annot_path:
    CHR0 = list(CHR_set_annot)[0]
    col_list = list(
        ldspec.util.read_annot(dic_annot_path[annot_name][CHR0], nrows=5)
    )
    for CHR in CHR_set_annot:
        temp_df = ldspec.util.read_annot(
            dic_annot_path[annot_name][CHR], nrows=5
        )
        err_msg = "%s : columns mismatch between CHR%d and CHR%d" % (
            annot_name,
            CHR0,
            CHR,
        )
        assert list(temp_df) == col_list, err_msg
    print("    %s: columms match for all CHRs. Containing" % annot_name)
    temp_str = ",".join([x for x in col_list if x.startswith("AN:")])
    print("        %s" % temp_str)

# Check: pannots have the same shape as pvar file
for annot_name in dic_pannot_path:
    CHR = np.random.choice(list(CHR_set_annot), size=1)[0]
    mat_G = ldspec.util.read_pannot_mat(dic_pannot_path[annot_name][CHR])
    err_msg = "(%s, CHR%d) : n_snp=%d, mismatch with --pgen_file" % (
        annot_name,
        CHR,
        mat_G.shape[0],
    )
    assert mat_G.shape[0] == dic_data[CHR]["pvar"].shape[0], err_msg
    print( 
        "    %s (%d CHRs) : CHR%d dimension matches with .pvar"
        % (annot_name, len(dic_pannot_path[annot_name]), CHR)
    )

# Loading --annot_file
    Detected 22 CHRs for all files: 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
    AN:baseline_165annots: columms match for all CHRs. Containing
        AN:Backgrd_Selection_Stat_common,AN:Backgrd_Selection_Stat_lf,AN:BivFlnk.flanking.500_common,AN:BivFlnk.flanking.500_lf,AN:BivFlnk_common,AN:BivFlnk_lf,AN:CADD_g20_common,AN:CADD_g20_lf,AN:CTCF_Hoffman.flanking.500_common,AN:CTCF_Hoffman.flanking.500_lf,AN:CTCF_Hoffman_common,AN:CTCF_Hoffman_lf,AN:Coding_UCSC.flanking.500_common,AN:Coding_UCSC.flanking.500_lf,AN:Coding_UCSC_common,AN:Coding_UCSC_lf,AN:Conserved_LindbladToh.flanking.500_common,AN:Conserved_LindbladToh.flanking.500_lf,AN:Conserved_LindbladToh_common,AN:Conserved_LindbladToh_lf,AN:Conserved_Mammal_phastCons46way.flanking.500_common,AN:Conserved_Mammal_phastCons46way.flanking.500_lf,AN:Conserved_Mammal_phastCons46way_common,AN:Conserved_Mammal_phastCons46way_lf,AN:Conserved_Primate_phastCons46way.flanking.500_common,AN:Conserved_Primate

    pAN:Repressed_Hoffman_proxy_0_100_ld_p0_p100_maf_common_block (22 CHRs) : CHR1 dimension matches with .pvar
    pAN:Repressed_Hoffman_proxy_0_100_ld_p0_p100_maf_lf_block (22 CHRs) : CHR6 dimension matches with .pvar
    pAN:SuperEnhancer_Hnisz_proxy_0_100_ld_p0_p100_maf_common_block (22 CHRs) : CHR2 dimension matches with .pvar
    pAN:SuperEnhancer_Hnisz_proxy_0_100_ld_p0_p100_maf_lf_block (22 CHRs) : CHR3 dimension matches with .pvar
    pAN:Transcribed_Hoffman_proxy_0_100_ld_p0_p100_maf_common_block (22 CHRs) : CHR5 dimension matches with .pvar
    pAN:Transcribed_Hoffman_proxy_0_100_ld_p0_p100_maf_lf_block (22 CHRs) : CHR1 dimension matches with .pvar
    pAN:H3K27ac_Hnisz_proxy_0_100_ld_n100_p0_maf_common_block (22 CHRs) : CHR4 dimension matches with .pvar
    pAN:H3K27ac_Hnisz_proxy_0_100_ld_n100_p0_maf_lf_block (22 CHRs) : CHR6 dimension matches with .pvar
    pAN:H3K27ac_PGC2_proxy_0_100_ld_n100_p0_maf_common_block (22 CHRs) : CHR5 dimension matches with .pvar
    pAN:H3K27

    pAN:H3K9ac_Trynka_proxy_0_1000_ld_n100_p0_maf_lf_block (22 CHRs) : CHR3 dimension matches with .pvar
    pAN:Intron_UCSC_proxy_0_1000_ld_n100_p0_maf_common_block (22 CHRs) : CHR5 dimension matches with .pvar
    pAN:Intron_UCSC_proxy_0_1000_ld_n100_p0_maf_lf_block (22 CHRs) : CHR12 dimension matches with .pvar
    pAN:Promoter_UCSC_proxy_0_1000_ld_n100_p0_maf_common_block (22 CHRs) : CHR1 dimension matches with .pvar
    pAN:Promoter_UCSC_proxy_0_1000_ld_n100_p0_maf_lf_block (22 CHRs) : CHR2 dimension matches with .pvar
    pAN:Repressed_Hoffman_proxy_0_1000_ld_n100_p0_maf_common_block (22 CHRs) : CHR15 dimension matches with .pvar
    pAN:Repressed_Hoffman_proxy_0_1000_ld_n100_p0_maf_lf_block (22 CHRs) : CHR20 dimension matches with .pvar
    pAN:SuperEnhancer_Hnisz_proxy_0_1000_ld_n100_p0_maf_common_block (22 CHRs) : CHR19 dimension matches with .pvar
    pAN:SuperEnhancer_Hnisz_proxy_0_1000_ld_n100_p0_maf_lf_block (22 CHRs) : CHR18 dimension matches with .pvar
    pAN:TFBS_ENCOD

### Update file 

In [5]:
# update annot file 
for CHR in dic_data:
    for annot_name in dic_annot_path:
        # Check if file exists 
        out_file = OUT_FOLDER+os.sep+dic_annot_path[annot_name][CHR].split(os.sep)[-1]
        if os.path.exists(out_file):
            print('Exist', out_file)
            continue
        temp_df = ldspec.util.read_annot(dic_annot_path[annot_name][CHR])
        # temp_df_new
        temp_df_new = dic_data_new[CHR]['pvar'][["CHR", "SNP", "BP"]].copy()
        for AN in [x for x in temp_df if x.startswith("AN:")]:
            temp_dic = {x: y for x, y in zip(temp_df["SNP"], temp_df[AN]) if y != 0}
            temp_df_new[AN] = [temp_dic[x] if x in temp_dic else 0 for x in temp_df_new['SNP']]
            temp_df_new[AN] = temp_df_new[AN].astype(temp_df[AN].dtype)
        # Write file 
        ldspec.util.write_annot(temp_df_new, out_file)
        print('Written', temp_df_new.shape, out_file)

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr1.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr2.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr3.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr4.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr5.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr6.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr7.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr8.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr9.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr10.annot.gz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr11.annot.gz
Exist /n/groups/price/martin/L

In [6]:
# update pannot file 
for CHR in dic_data:
    for pAN in dic_pannot_path:
        # Check if file exists 
        out_file = OUT_FOLDER+os.sep+dic_pannot_path[pAN][CHR].split(os.sep)[-1]
        if os.path.exists(out_file):
            print('Exist', out_file)
            continue
            
        mat_G = ldspec.util.read_pannot_mat(dic_pannot_path[pAN][CHR])        
        # snp_pair_list
        v1,v2 = mat_G.nonzero()
        ind_select = v1<v2
        snp_pair_list = []
        for i,j in zip(v1[ind_select],v2[ind_select]):
            snp_pair_list.append((dic_data[CHR]['pvar']['SNP'][i], dic_data[CHR]['pvar']['SNP'][j]))
        # kill pairs not in the new file 
        temp_set = set(dic_data_new[CHR]['pvar']['SNP'])
        snp_pair_list = [x for x in snp_pair_list if (x[0] in temp_set) & (x[1] in temp_set)]
        # Write pannot
        ldspec.util.write_pannot_mat(
            snp_pair_list, list(dic_data_new[CHR]['pvar']['SNP']), 
            out_file.replace('.pannot_mat.npz', '')
        )
        print('Written', len(snp_pair_list), out_file)        

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_p0_p100.maf_common_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_p0_p100.maf_lf_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_100_1000.ld_p0_p100.maf_common_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_100_1000.ld_p0_p100.maf_lf_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_p0_p100.maf_common_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_p0_p100.maf_lf_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_n100_p0.maf_common_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_n100_p0.maf_lf_block.c1.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_100_1000.ld_n100_p0.maf_common_bloc

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DHS_peaks_Trynka.proxy_0_1000.ld_n100_p0.maf_common_block.c2.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DHS_peaks_Trynka.proxy_0_1000.ld_n100_p0.maf_lf_block.c2.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Enhancer_Hoffman.proxy_0_1000.ld_n100_p0.maf_common_block.c2.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Enhancer_Hoffman.proxy_0_1000.ld_n100_p0.maf_lf_block.c2.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/FetalDHS_Trynka.proxy_0_1000.ld_n100_p0.maf_common_block.c2.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/FetalDHS_Trynka.proxy_0_1000.ld_n100_p0.maf_lf_block.c2.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_Hnisz.proxy_0_1000.ld_n100_p0.maf_common_block.c2.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_Hnisz.proxy_0_1000.ld_n100_p0.maf_lf_

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me1_peaks_Trynka.proxy_0_1000.ld_p0_p100.maf_lf_block.c3.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me3_Trynka.proxy_0_1000.ld_p0_p100.maf_common_block.c3.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me3_Trynka.proxy_0_1000.ld_p0_p100.maf_lf_block.c3.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K9ac_Trynka.proxy_0_1000.ld_p0_p100.maf_common_block.c3.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K9ac_Trynka.proxy_0_1000.ld_p0_p100.maf_lf_block.c3.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Intron_UCSC.proxy_0_1000.ld_p0_p100.maf_common_block.c3.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Intron_UCSC.proxy_0_1000.ld_p0_p100.maf_lf_block.c3.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Promoter_UCSC.proxy_0_1000.ld_p0_p100.maf_common_block.c3.pa

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_n100_p0.maf_lf_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exon.ld_p0_p100.maf_common_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exon.ld_p0_p100.maf_lf_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exonic_gene.ld_p0_p100.maf_common_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exonic_gene.ld_p0_p100.maf_lf_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/cS2G_promoter.ld_p0_p100.maf_common_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/cS2G_promoter.ld_p0_p100.maf_lf_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/protein_domain.ld_p0_p100.maf_common_block.c4.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/protein_domain.ld_p0_p100.maf_lf_block.c4.pannot_mat.npz
Ex

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/gene.ld_n100_p0.maf_common_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/gene.ld_n100_p0.maf_lf_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_Hnisz.proxy_0_100.ld_p0_p100.maf_common_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_Hnisz.proxy_0_100.ld_p0_p100.maf_lf_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_PGC2.proxy_0_100.ld_p0_p100.maf_common_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_PGC2.proxy_0_100.ld_p0_p100.maf_lf_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me1_Trynka.proxy_0_100.ld_p0_p100.maf_common_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me1_Trynka.proxy_0_100.ld_p0_p100.maf_lf_block.c5.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/a

...

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DGF_ENCODE.proxy_0_1000.ld_n100_p0.maf_lf_block.c17.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DHS_Trynka.proxy_0_1000.ld_n100_p0.maf_common_block.c17.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DHS_Trynka.proxy_0_1000.ld_n100_p0.maf_lf_block.c17.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DHS_peaks_Trynka.proxy_0_1000.ld_n100_p0.maf_common_block.c17.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DHS_peaks_Trynka.proxy_0_1000.ld_n100_p0.maf_lf_block.c17.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Enhancer_Hoffman.proxy_0_1000.ld_n100_p0.maf_common_block.c17.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Enhancer_Hoffman.proxy_0_1000.ld_n100_p0.maf_lf_block.c17.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/FetalDHS_Trynka.proxy_0_1000.ld_n100_p0.maf_common_bloc

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_PGC2.proxy_0_1000.ld_p0_p100.maf_lf_block.c18.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me1_Trynka.proxy_0_1000.ld_p0_p100.maf_common_block.c18.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me1_Trynka.proxy_0_1000.ld_p0_p100.maf_lf_block.c18.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me1_peaks_Trynka.proxy_0_1000.ld_p0_p100.maf_common_block.c18.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me1_peaks_Trynka.proxy_0_1000.ld_p0_p100.maf_lf_block.c18.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me3_Trynka.proxy_0_1000.ld_p0_p100.maf_common_block.c18.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K4me3_Trynka.proxy_0_1000.ld_p0_p100.maf_lf_block.c18.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K9ac_Trynka.proxy_0_1000.ld_p0_p100.maf

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exonic_gene.ld_n100_p0.maf_common_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exonic_gene.ld_n100_p0.maf_lf_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/cS2G_promoter.ld_n100_p0.maf_common_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/cS2G_promoter.ld_n100_p0.maf_lf_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/protein_domain.ld_n100_p0.maf_common_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/protein_domain.ld_n100_p0.maf_lf_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/gene.ld_n100_p0.maf_common_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/gene.ld_n100_p0.maf_lf_block.c20.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/H3K27ac_Hnisz.proxy_0_100.ld_p0_p100.maf_common_block

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_p0_p100.maf_lf_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_n100_p0.maf_common_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_n100_p0.maf_lf_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_100_1000.ld_n100_p0.maf_common_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_100_1000.ld_n100_p0.maf_lf_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_n100_p0.maf_common_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_n100_p0.maf_lf_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exon.ld_p0_p100.maf_common_block.c21.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/exon.ld_p0_p100.maf_lf_block.c21.pa

Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Transcribed_Hoffman.proxy_0_1000.ld_p0_p100.maf_lf_block.c22.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Vahedi_Tcell_SE.proxy_0_1000.ld_p0_p100.maf_common_block.c22.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Vahedi_Tcell_SE.proxy_0_1000.ld_p0_p100.maf_lf_block.c22.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Vahedi_Tcell_TE.proxy_0_1000.ld_p0_p100.maf_common_block.c22.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/Vahedi_Tcell_TE.proxy_0_1000.ld_p0_p100.maf_lf_block.c22.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DGF_ENCODE.proxy_0_1000.ld_n100_p0.maf_common_block.c22.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DGF_ENCODE.proxy_0_1000.ld_n100_p0.maf_lf_block.c22.pannot_mat.npz
Exist /n/groups/price/martin/LDSPEC_data/annot_rep287K/DHS_Trynka.proxy_0_1000.ld_n100_p0.maf_common_bloc

In [7]:
# update ANNOT_FILE
with open(OUT_ANNOT_FILE, 'w') as fout:
    with open(ANNOT_FILE, "r") as fin:        
        for line in fin:
            line = line.strip()
            line = OUT_FOLDER+os.sep+line.split(os.sep)[-1]
            fout.write('%s\n' % line)
            print(line.replace('@','22'))

/n/groups/price/martin/LDSPEC_data/annot_rep287K/baseline_165annots_chr22.annot.gz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_p0_p100.maf_common_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_p0_p100.maf_lf_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_100_1000.ld_p0_p100.maf_common_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_100_1000.ld_p0_p100.maf_lf_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_p0_p100.maf_common_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_1000_10000.ld_p0_p100.maf_lf_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_n100_p0.maf_common_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/proxy_0_100.ld_n100_p0.maf_lf_block.c22.pannot_mat.npz
/n/groups/price/martin/LDSPEC_data/annot_rep287K/prox

In [8]:
# check all files
with open(OUT_ANNOT_FILE, "r") as fin: 
    for line in fin:
        line = line.strip()
        for CHR in range(1,23):
            size = os.path.getsize(line.replace('@','%s'%CHR)) / 1024
            if size < 1:
                print('%0.1fMB'%size, line)
print('Finish!')

Finish!


In [9]:
# # check all files
# with open(
#     '/home/jz286/WES_analysis/LDSPEC/experiments/job.simulation_chimp/reg_annot_file.sim.rv1.rep287K.txt', 
#     "r") as fin: 
#     for line in fin:
#         line = line.strip()
#         for CHR in range(1,23):
#             size = os.path.getsize(line.replace('@','%s'%CHR)) / 1024
#             if size < 1:
#                 print('%0.1fMB'%size, line)